<a href="https://colab.research.google.com/github/inekeyuliam/data-science-classification-python/blob/master/Logistic_Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import pylab as pl
from sklearn import preprocessing
%matplotlib inline
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv('framingham.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [31]:
df.dtypes
missingvalues = df.isnull()
missingvalues.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [32]:
for column in missingvalues.columns.values.tolist():
  print(column)
  print(missingvalues[column].value_counts())
  print("")

male
False    4238
Name: male, dtype: int64

age
False    4238
Name: age, dtype: int64

education
False    4133
True      105
Name: education, dtype: int64

currentSmoker
False    4238
Name: currentSmoker, dtype: int64

cigsPerDay
False    4209
True       29
Name: cigsPerDay, dtype: int64

BPMeds
False    4185
True       53
Name: BPMeds, dtype: int64

prevalentStroke
False    4238
Name: prevalentStroke, dtype: int64

prevalentHyp
False    4238
Name: prevalentHyp, dtype: int64

diabetes
False    4238
Name: diabetes, dtype: int64

totChol
False    4188
True       50
Name: totChol, dtype: int64

sysBP
False    4238
Name: sysBP, dtype: int64

diaBP
False    4238
Name: diaBP, dtype: int64

BMI
False    4219
True       19
Name: BMI, dtype: int64

heartRate
False    4237
True        1
Name: heartRate, dtype: int64

glucose
False    3850
True      388
Name: glucose, dtype: int64

TenYearCHD
False    4238
Name: TenYearCHD, dtype: int64



In [50]:
df['education'].value_counts().idxmax()
df['education'].replace(np.nan, 1.0,inplace=True)
df['cigsPerDay'].value_counts().idxmax()
df['cigsPerDay'].replace(np.nan, 0.0, inplace=True)
df['BPMeds'].value_counts().idxmax()
df['BPMeds'].replace(np.nan, 0.0, inplace=True)
avgchol = df['totChol'].mean()
df['totChol'].replace(np.nan, avgchol, inplace=True)
avgbmi = df['BMI'].mean()
df['BMI'].replace(np.nan, avgbmi, inplace=True)
avgheart = df['heartRate'].mean()
df['heartRate'].replace(np.nan, avgheart, inplace=True)
avgglu = df['glucose'].mean()
df['glucose'].replace(np.nan, avgglu, inplace=True)

In [51]:
df.corr()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
male,1.000000,-0.028979,0.013299,0.197596,0.315630,-0.051545,-0.004546,0.005313,0.015708,-0.069974,-0.035989,0.057933,0.081506,-0.116601,0.005818,0.088428
age,-0.028979,1.000000,-0.164809,-0.213748,-0.191847,0.120955,0.057655,0.307194,0.101258,0.260270,0.394302,0.206104,0.135283,-0.012819,0.116850,0.225256
education,0.013299,-0.164809,1.000000,0.019403,0.009905,-0.010149,-0.032881,-0.078807,-0.038147,-0.023534,-0.126197,-0.062732,-0.139487,-0.049604,-0.034299,-0.052812
currentSmoker,0.197596,-0.213748,0.019403,1.000000,0.760239,-0.048358,-0.032988,-0.103260,-0.044295,-0.046285,-0.130230,-0.107746,-0.167276,0.062348,-0.054157,0.019456
cigsPerDay,0.315630,-0.191847,0.009905,0.760239,1.000000,-0.044675,-0.032244,-0.065046,-0.036150,-0.026816,-0.088375,-0.056687,-0.092332,0.073866,-0.056650,0.058859
BPMeds,-0.051545,0.120955,-0.010149,-0.048358,-0.044675,1.000000,0.114609,0.258697,0.051394,0.078712,0.251503,0.192356,0.099647,0.015139,0.048856,0.086417
prevalentStroke,-0.004546,0.057655,-0.032881,-0.032988,-0.032244,0.114609,1.000000,0.074830,0.006949,0.000067,0.057009,0.045190,0.024840,-0.017676,0.018055,0.061810
prevalentHyp,0.005313,0.307194,-0.078807,-0.103260,-0.065046,0.258697,0.074830,1.000000,0.077808,0.163041,0.696755,0.615751,0.300572,0.147222,0.082924,0.177603
diabetes,0.015708,0.101258,-0.038147,-0.044295,-0.036150,0.051394,0.006949,0.077808,1.000000,0.040092,0.111283,0.050329,0.086250,0.048993,0.605705,0.097317
totChol,-0.069974,0.260270,-0.023534,-0.046285,-0.026816,0.078712,0.000067,0.163041,0.040092,1.000000,0.207609,0.163903,0.114789,0.090676,0.044583,0.081624


In [52]:
X = np.asarray(df[['male','age','currentSmoker','cigsPerDay','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol','sysBP','diaBP','BMI','heartRate','glucose']])
X[0:5]

array([[  1.  ,  39.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
        195.  , 106.  ,  70.  ,  26.97,  80.  ,  77.  ],
       [  0.  ,  46.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
        250.  , 121.  ,  81.  ,  28.73,  95.  ,  76.  ],
       [  1.  ,  48.  ,   1.  ,  20.  ,   0.  ,   0.  ,   0.  ,   0.  ,
        245.  , 127.5 ,  80.  ,  25.34,  75.  ,  70.  ],
       [  0.  ,  61.  ,   1.  ,  30.  ,   0.  ,   0.  ,   1.  ,   0.  ,
        225.  , 150.  ,  95.  ,  28.58,  65.  , 103.  ],
       [  0.  ,  46.  ,   1.  ,  23.  ,   0.  ,   0.  ,   0.  ,   0.  ,
        285.  , 130.  ,  84.  ,  23.1 ,  85.  ,  85.  ]])

In [53]:
y=np.asarray(df['TenYearCHD'])
y[0:5]

array([0, 0, 0, 1, 0])

In [54]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 1.1531919 , -1.23495068, -0.98827076, -0.75132224, -0.17361158,
        -0.07703255, -0.67110093, -0.16247659, -0.94134554, -1.19590711,
        -1.08262515,  0.28694273,  0.34274444, -0.21751656],
       [-0.86715836, -0.41825733, -0.98827076, -0.75132224, -0.17361158,
        -0.07703255, -0.67110093, -0.16247659,  0.29959495, -0.51518725,
        -0.15898843,  0.71932499,  1.59027451, -0.26131108],
       [ 1.1531919 , -0.18491638,  1.01186844,  0.92920959, -0.17361158,
        -0.07703255, -0.67110093, -0.16247659,  0.18678218, -0.22020864,
        -0.24295541, -0.11350221, -0.07309892, -0.52407818],
       [-0.86715836,  1.33179984,  1.01186844,  1.76947551, -0.17361158,
        -0.07703255,  1.49008884, -0.16247659, -0.26446891,  0.80087114,
         1.0165492 ,  0.68247423, -0.90478564,  0.92114088],
       [-0.86715836, -0.41825733,  1.01186844,  1.18128937, -0.17361158,
        -0.07703255, -0.67110093, -0.16247659,  1.08928436, -0.10675533,
         0.09291249, -0.66

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=4)
print("train set : ", X_train.shape, y_train.shape)
print("test set : ", X_test.shape, y_test.shape)

train set :  (3390, 14) (3390,)
test set :  (848, 14) (848,)


In [57]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
lr

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
yhat = lr.predict(X_test)
yhat

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [60]:
yhat_prob = lr.predict_proba(X_test)
yhat_prob

array([[0.88700844, 0.11299156],
       [0.68390048, 0.31609952],
       [0.88134417, 0.11865583],
       ...,
       [0.8143512 , 0.1856488 ],
       [0.70270899, 0.29729101],
       [0.82344672, 0.17655328]])

In [64]:
from sklearn.metrics import confusion_matrix, classification_report,log_loss
print(confusion_matrix(y_test,yhat))

[[712   5]
 [124   7]]


In [62]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.85      0.99      0.92       717
           1       0.58      0.05      0.10       131

    accuracy                           0.85       848
   macro avg       0.72      0.52      0.51       848
weighted avg       0.81      0.85      0.79       848



In [63]:

from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,yhat)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


0.847877358490566

In [65]:
log_loss(y_test,yhat_prob)

0.3967944161264772